In [7]:
import os
from dotenv import load_dotenv

# Dynamically get the path to the .env file for Jupyter/IPython
current_directory = os.getcwd()
env_path = os.path.join(current_directory, 'API_KEY.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

# Access API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NINJA_API_KEY = os.getenv("NINJA_API_KEY")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")


In [10]:
import requests
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import gradio as gr

# API URLs
NINJA_API_URL = "https://api.api-ninjas.com/v1/earningstranscript"
ALPHA_VANTAGE_API_URL = "https://www.alphavantage.co/query"
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"

# Function to fetch earnings data
def fetch_earnings_data(ticker):
    params = {"function": "EARNINGS", "symbol": ticker, "apikey": ALPHA_VANTAGE_API_KEY}
    try:
        response = requests.get(ALPHA_VANTAGE_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get("quarterlyEarnings")
    except Exception as e:
        return f"Error fetching earnings data: {e}"

# Function to extract earnings for the specified period
def extract_earnings_for_period(data, year, quarter):
    for record in data:
        fiscal_date = datetime.strptime(record["fiscalDateEnding"], '%Y-%m-%d')
        fiscal_year = fiscal_date.year
        fiscal_quarter = (fiscal_date.month - 1) // 3 + 1
        if fiscal_year == int(year) and fiscal_quarter == int(quarter):
            earnings_call_date = fiscal_date + timedelta(days=30)
            return {
                'earningsCallDate': earnings_call_date.strftime('%Y-%m-%d'),
                'reportedEPS': record.get("reportedEPS"),
                'estimatedEPS': record.get("estimatedEPS")
            }
    return None

# Function to fetch transcript
def fetch_transcript(ticker, year, quarter):
    headers = {"X-Api-Key": NINJA_API_KEY}
    params = {"ticker": ticker, "year": year, "quarter": quarter}
    try:
        response = requests.get(NINJA_API_URL, headers=headers, params=params)
        response.raise_for_status()
        return response.json().get("transcript")
    except Exception as e:
        return None

# Function to analyze sentiment
def analyze_sentiment_with_openai(text):
    headers = {"Authorization": f"Bearer {OPENAI_API_KEY}", "Content-Type": "application/json"}
    try:
        payload = {
            "model": "gpt-3.5-turbo",
            "messages": [
                {"role": "system", "content": "You are a sentiment analysis assistant."},
                {"role": "user", "content": f"Analyze sentiment: {text}"}
            ]
        }
        response = requests.post(OPENAI_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    except Exception as e:
        return "Unknown"

# Function to calculate growth
def calculate_growth(stock_data, start_date, end_date):
    try:
        start_index = stock_data.index.get_indexer([start_date], method='nearest')[0]
        end_index = stock_data.index.get_indexer([end_date], method='nearest')[0]
        start_price = float(stock_data.iloc[start_index]["Close"])
        end_price = float(stock_data.iloc[end_index]["Close"])
        return ((end_price - start_price) / start_price) * 100
    except Exception as e:
        return None

# Function to make recommendations
def make_recommendation(sentiment, reported_eps, estimated_eps):
    eps_difference = float(reported_eps) - float(estimated_eps)
    if sentiment == "Positive" and eps_difference > 0:
        return "Buy", "The sentiment is positive and the company exceeded EPS expectations. Consider buying."
    elif sentiment == "Negative" or eps_difference < 0:
        return "Sell", "The sentiment is negative, or the company missed EPS expectations. Consider selling."
    else:
        return "Hold", "The sentiment is neutral, and there are no strong EPS surprises. Hold the stock."

# Main function to process user inputs
def process_data(ticker, year, quarter):
    # Fetch earnings data
    earnings_data = fetch_earnings_data(ticker)
    if not earnings_data:
        return "Error fetching earnings data", None, None, None, None

    earnings_info = extract_earnings_for_period(earnings_data, year, quarter)
    if not earnings_info:
        return "Earnings info unavailable", None, None, None, None

    # Fetch transcript and analyze sentiment
    transcript = fetch_transcript(ticker, year, quarter)
    sentiment = "Unknown"
    if transcript:
        sentiment = analyze_sentiment_with_openai(transcript)

    # Fetch stock prices
    earnings_date = datetime.strptime(earnings_info['earningsCallDate'], '%Y-%m-%d')
    stock_data = yf.download(ticker, start=earnings_date - timedelta(days=10), end=earnings_date + timedelta(days=90))

    # Calculate growth
    growth_7_days = calculate_growth(stock_data, earnings_date - timedelta(days=7), earnings_date + timedelta(days=7))
    growth_1_month = calculate_growth(stock_data, earnings_date - timedelta(days=7), earnings_date + timedelta(days=30))
    growth_3_months = calculate_growth(stock_data, earnings_date - timedelta(days=7), earnings_date + timedelta(days=90))

    # Generate charts
    charts = []
    # Plot stock prices
    plt.figure(figsize=(10, 5))
    plt.plot(stock_data['Close'], label="Stock Price", linewidth=2)
    plt.axvline(x=earnings_date, color='red', linestyle='--', label="Earnings Call Date")
    plt.title("Stock Price Movement Around Earnings Call")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.tight_layout()
    plt.savefig("stock_price_chart.png")
    charts.append("stock_price_chart.png")

    # Plot growth
    plt.figure(figsize=(8, 5))
    growth_data = {"7 Days": growth_7_days, "1 Month": growth_1_month, "3 Months": growth_3_months}
    plt.bar(growth_data.keys(), growth_data.values(), color='blue', alpha=0.7)
    plt.title("Growth Over Periods")
    plt.ylabel("Growth (%)")
    plt.tight_layout()
    plt.savefig("growth_chart.png")
    charts.append("growth_chart.png")

    # Plot EPS comparison
    plt.figure(figsize=(5, 5))
    labels = ['Reported EPS', 'Estimated EPS']
    values = [float(earnings_info['reportedEPS']), float(earnings_info['estimatedEPS'])]
    plt.bar(labels, values, color=['blue', 'black'], alpha=0.7)
    plt.title("EPS Comparison")
    plt.tight_layout()
    plt.savefig("eps_chart.png")
    charts.append("eps_chart.png")

    # Recommendation
    recommendation, rationale = make_recommendation(
        sentiment, earnings_info['reportedEPS'], earnings_info['estimatedEPS']
    )

    return (
        f"Sentiment: {sentiment}\n"
        f"Earnings Call Date: {earnings_info['earningsCallDate']}\n"
        f"Reported EPS: {earnings_info['reportedEPS']}\n"
        f"Estimated EPS: {earnings_info['estimatedEPS']}\n"
        f"Growth (7 Days): {growth_7_days:.2f}%\n"
        f"Growth (1 Month): {growth_1_month:.2f}%\n"
        f"Growth (3 Months): {growth_3_months:.2f}%\n"
        f"Recommendation: {recommendation}\n"
        f"Rationale: {rationale}",
        charts[0],
        charts[1],
        charts[2],
    )

# Gradio Interface
iface = gr.Interface(
    fn=process_data,
    inputs=[
        gr.Textbox(label="Stock Ticker (e.g., AAPL)", placeholder="Enter the stock ticker"),
        gr.Textbox(label="Year (e.g., 2023)", placeholder="Enter the year of analysis"),
        gr.Textbox(label="Quarter (1-4)", placeholder="Enter the quarter number"),
    ],
    outputs=[
        gr.Textbox(label="Analysis Results", lines=8),
        gr.Image(label="Stock Price Chart"),
        gr.Image(label="Growth Chart"),
        gr.Image(label="EPS Comparison Chart"),
    ],
    title="📈 Earnings Analysis Dashboard",
    description=(
        "Analyze stock earnings performance, sentiment, and growth trends. "
        "Input a stock ticker, year, and quarter to generate insights with charts and recommendations."
    ),
    theme="compact",
    css="""
        .gradio-container {
            font-family: 'Arial', sans-serif;
        }
        h1 {
            text-align: center;
            color: #333;
        }https://github.com/Homebrew/brew
    """,
)

iface.launch(share=True)


/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/gradio/blocks.py:982: UserWarning: Cannot load compact. Caught Exception: module 'huggingface_hub.utils' has no attribute '_errors'
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://82ce19ab8938d24375.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[*********************100%***********************]  1 of 1 completed
